# Climate Change Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sci

In [9]:
#data
datadir = './data/'
global_mean_temp = pd.read_csv(datadir + 'global-mean-temp.csv')
monthly_c02 = pd.read_csv(datadir + 'co2_mm_mlo.csv')

In [10]:
global_mean_temp.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.17,-0.23,-0.08,-0.15,-0.09,-0.20,-0.17,-0.09,-0.13,-0.22,-.21,-.16,-.16,***,***,-0.11,-0.15,-.19
1,1881,-0.18,-0.13,0.04,0.06,0.07,-0.17,0.02,-0.02,-0.14,-0.20,-.17,-.05,-.07,-.08,-.16,0.06,-0.06,-.17
2,1882,0.18,0.15,0.06,-0.15,-0.13,-0.22,-0.15,-0.06,-0.13,-0.22,-.15,-.34,-.10,-.07,.09,-0.08,-0.14,-.17
3,1883,-0.28,-0.36,-0.11,-0.17,-0.16,-0.07,-0.05,-0.12,-0.20,-0.10,-.22,-.10,-.16,-.18,-.33,-0.15,-0.08,-.17
4,1884,-0.12,-0.06,-0.35,-0.39,-0.33,-0.35,-0.31,-0.26,-0.26,-0.24,-.32,-.30,-.27,-.26,-.09,-0.36,-0.31,-.28


In [11]:
monthly_c02.head()

,year,month,decimal date,average,interpolated,trend,ndays
1958,3,1958.2027,315.70,314.43,-1,-9.99,-0.99
1958,4,1958.2877,317.45,315.16,-1,-9.99,-0.99
1958,5,1958.3699,317.51,314.71,-1,-9.99,-0.99
1958,6,1958.4548,317.24,315.14,-1,-9.99,-0.99
1958,7,1958.5370,315.86,315.18,-1,-9.99,-0.99
